In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

this an example on how to do FCI fro the 4 index integrals, or the 2 electron integrals. 

In [2]:
def Add_Block_Matrices(Paa, Pbb):
    Pab = np.zeros(Paa.shape)
    Pba = np.zeros(Paa.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# basic setup of a molecule
element = "He"
mol = gto.Mole()
mol.atom = f'''{element}  0 0 0'''
mol.basis = '631-g'
mol.build()
eri_ao = mol.intor('int2e')

In [4]:
# run Hartree Fock
mf = scf.UHF(mol)
mf.kernel()

converged SCF energy = -2.85516042615445  <S^2> = -4.4408921e-16  2S+1 = 1


np.float64(-2.8551604261544465)

In [18]:
# Harvest HF quantities
# M = h1.shape[0]
h1 = mf.get_hcore()
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[0,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b

eri_h2_aa = eri_ao.copy()
eri_h2_bb = eri_ao.copy()
eri_h2_ab = eri_ao.copy()


for i in range(4):
  eri_h2_aa = np.tensordot(eri_h2_aa, C_a, axes=1).transpose(3, 0, 1, 2)
  eri_h2_bb = np.tensordot(eri_h2_bb, C_b, axes=1).transpose(3, 0, 1, 2)

eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)
    


In [19]:
# and this is the purpose of this script print(
e_1, ci = fci.direct_uhf.kernel((h1_a, h1_b), (eri_h2_aa, eri_h2_ab, eri_h2_bb), h1.shape[0], mol.nelec, ecore=mf.energy_nuc())

In [7]:
e_2, ci = fci.direct_spin0.kernel(h1_a, eri_h2_aa, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())

In [20]:
print(e_1, e_2)

-2.870162138900823 -2.8701621389008234
